Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации - выбить auc выше 0.95

In [1]:
import pandas as pd

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn import model_selection

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score 

from gensim.models import Word2Vec

from tqdm import tqdm_notebook

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to C:\Users\marchenko-
[nltk_data]     i-u\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [114]:
df =  pd.read_excel("отзывы за лето.xls")

In [115]:
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [116]:
df.shape

(20659, 3)

In [117]:
df_train = df.loc[:4131]
df_val = df.loc[4132:]

In [118]:
df_train.shape, df_val.shape

((4132, 3), (16527, 3))

In [184]:
max_words = 200
max_len = 40

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [185]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    txt = [word for word in txt if len(word)>1]
    
    txt = [word for word in txt if word.isalnum()]
    
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [186]:
df_train['Content'].head(10)

0                                        it just works
1    целое удобнона приложениеиз минус большой дост...
2                                              отлично
3    зависать работа антивирус ранее пользоваться н...
4                               удобно работать быстро
5                                         удобно норма
6                                   удобный приложение
7                                           устраивать
8    работать чётко отличие банкомат вечно зависать...
9                                                     
Name: Content, dtype: object

In [187]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [188]:
tokens = word_tokenize(train_corpus)

In [189]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [200]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)] 

In [201]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [202]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [203]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [204]:
x_train

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ..., 105,  93,  12],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   4,   1],
       [  0,   0,   0, ...,   0,   0,   4]])

In [205]:
df_train["Rating"].unique()

array([5, 4, 2, 3, 1], dtype=int64)

In [206]:
num_classes = 6
y_train = keras.utils.to_categorical(df_train["Rating"], num_classes)
y_val = keras.utils.to_categorical(df_val["Rating"], num_classes)

In [248]:
from keras.layers import GlobalAveragePooling1D

In [261]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [262]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics='AUC')

In [263]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='auc', mode="auto")  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])

Epoch 1/20
8/8 [==============================] - 1s 156ms/step - loss: 1.6735 - auc: 0.8265 - val_loss: 1.5266 - val_auc: 0.8318
Epoch 2/20
8/8 [==============================] - 1s 92ms/step - loss: 1.3690 - auc: 0.8478 - val_loss: 1.2102 - val_auc: 0.8386
Epoch 3/20
8/8 [==============================] - 1s 94ms/step - loss: 1.0458 - auc: 0.8551 - val_loss: 1.0944 - val_auc: 0.8463
Epoch 4/20
8/8 [==============================] - 1s 92ms/step - loss: 0.9722 - auc: 0.8788 - val_loss: 1.0742 - val_auc: 0.9068
Epoch 5/20
8/8 [==============================] - 1s 94ms/step - loss: 0.9150 - auc: 0.9076 - val_loss: 0.9697 - val_auc: 0.9182
Epoch 6/20
8/8 [==============================] - 1s 90ms/step - loss: 0.8671 - auc: 0.9275 - val_loss: 0.9264 - val_auc: 0.9256
Epoch 7/20
8/8 [==============================] - 1s 91ms/step - loss: 0.8271 - auc: 0.9359 - val_loss: 0.8875 - val_auc: 0.9363
Epoch 8/20
8/8 [==============================] - 1s 93ms/step - loss: 0.7784 - auc: 0.9471 - va

In [264]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

33/33 [==============================] - 1s 33ms/step - loss: 0.7317 - auc: 0.9450


Test score: 0.7316719889640808
Test accuracy: 0.9449595212936401


2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [59]:
modelW2V = Word2Vec(sentences=df_train['Content'].apply(str.split), size=40, window=5, min_count=1)

In [60]:
vect_idf = TfidfVectorizer()
vect_idf.fit_transform(df_train['Content'])
tfidf = dict(zip(vect_idf.get_feature_names(), vect_idf.idf_))

In [61]:
def get_vect_mean(txt):
    vector_w2v = np.zeros(40)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V:
            vector_w2v += modelW2V[wrd]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

In [62]:
arr_vect = []
for txt in tqdm_notebook(df_train['Content']):
    arr_vect.append(get_vect_mean(txt))
    
arr_vect_valid = []
for txt in tqdm_notebook(df_val['Content']):
    arr_vect_valid.append(get_vect_mean(txt))
    
train_w2v = np.asarray(arr_vect)    
valid_w2v = np.asarray(arr_vect_valid)

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [64]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics='AUC')

In [65]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)



history = model.fit(train_w2v, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])

Epoch 1/20
8/8 [==============================] - 1s 172ms/step - loss: 1.7093 - auc: 0.7795 - val_loss: 1.6187 - val_auc: 0.7290
Epoch 2/20
8/8 [==============================] - 1s 95ms/step - loss: 1.5363 - auc: 0.7878 - val_loss: 1.4569 - val_auc: 0.7290
Epoch 3/20
8/8 [==============================] - 1s 102ms/step - loss: 1.3633 - auc: 0.8015 - val_loss: 1.3525 - val_auc: 0.8604
Epoch 4/20
8/8 [==============================] - 1s 120ms/step - loss: 1.2433 - auc: 0.8784 - val_loss: 1.2462 - val_auc: 0.8512
Epoch 5/20
8/8 [==============================] - 1s 112ms/step - loss: 1.1062 - auc: 0.8747 - val_loss: 1.1512 - val_auc: 0.8512
Epoch 6/20
8/8 [==============================] - 1s 95ms/step - loss: 1.0182 - auc: 0.8705 - val_loss: 1.1214 - val_auc: 0.8309
Epoch 7/20
8/8 [==============================] - 1s 93ms/step - loss: 0.9795 - auc: 0.8725 - val_loss: 1.1024 - val_auc: 0.8493
Epoch 8/20
8/8 [==============================] - 1s 98ms/step - loss: 0.9557 - auc: 0.8745 -

In [66]:
score = model.evaluate(valid_w2v, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

33/33 [==============================] - 1s 36ms/step - loss: 1.0087 - auc: 0.8760


Test score: 1.0086997747421265
Test accuracy: 0.8759726285934448
